In [1]:
import pandas as pd
import numpy as np
import re
#import math
#from random import choices
#import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
#from matplotlib.colors import ListedColormap
#import statistics

In [2]:
import devanalyst.simulation.statics as S_

from devanalyst.simulation.businessObjects import WorkAssignments, ReleaseCycleContext, Ticket, UserStory
from devanalyst.simulation.simulationModels import BalancedAllocationModel, GreedyAllocationModel, ModelsConfig, \
Distribution, DefaultCostModel, QualityModel, DistributedLagQualityModel

import devanalyst.simulation.generateTimecards as timecard
#from devanalyst.simulation.generateTimecards import WorkAssignments


importing Jupyter notebook from c:\alejandro\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\statics.ipynb
importing Jupyter notebook from c:\alejandro\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\businessObjects.ipynb
importing Jupyter notebook from c:\alejandro\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\simulationModels.ipynb
importing Jupyter notebook from c:\alejandro\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\generateTimecards.ipynb


In [3]:
import devanalyst.simulation.tests.test_utils as tu_

importing Jupyter notebook from c:\alejandro\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\tests\test_utils.ipynb


In [4]:
import devanalyst.simulation.visualizations.simm_visuals as simm_visuals
import devanalyst.simulation.visualizations.timecard_visuals as tc_visuals

importing Jupyter notebook from c:\alejandro\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\visualizations\simm_visuals.ipynb
importing Jupyter notebook from c:\alejandro\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\visualizations\timecard_visuals.ipynb


<h1>Pilots</h1>

<h2>Bug inflow prototypes</h2>

In [6]:
def testTicketInflow(modelsConfig):
    RELEASE_DURATION = 60
    SPRINT_DURATION = 10
    
    tu_.loadTestResources()
    
    teams_df, stories_df, teamsRepo, storiesRepo, ticketsRepo = tu_.initTestData(tu_.DEV_DF, \
                                                                                           tu_.PM_DF, \
                                                                                           RELEASE_DURATION, \
                                                                                           SPRINT_DURATION, \
                                                                                          modelsConfig)
    TEAM_ID = 'Team A'
    SPRINT = 1    
    ctx = ReleaseCycleContext(TEAM_ID, teamsRepo, storiesRepo, ticketsRepo, SPRINT, SPRINT_DURATION)
    modelsConfig.context = ctx

    # Pretend all stories have been completed on the first sprint
    uss_list = []
    ids = ctx.storiesRepo.findIds()
    for userStoryId in ids:
        uss = ctx.teamsRepo.getUserStoryStatus(userStoryId)
        uss.percentAchieved = 1.0
        uss.planned = True
        uss.sprintPlanned = 1
        uss.sprintDelivered = 1
    
    # Now pretend we raverse the next 5 sprints, generating bugs
    bugs = []
    for i in range(5):
        ctx.sprint = SPRINT + i + 1
        qualityModel = modelsConfig.qualityModels[0]
        bugs.extend(qualityModel.findBugs(modelsConfig))
    return bugs

In [8]:
modelsConfig = ModelsConfig([DefaultCostModel(0.25)], [DistributedLagQualityModel()], GreedyAllocationModel()) 
modelsConfig.random.reset(271)
bugs = testTicketInflow(modelsConfig)
bugs_df = Ticket.build_bugs_df(bugs)
bugs_df[:6]

,Ticket Id,User Story Id,Estimated Cost,Effort to Date,Percent Achieved,Sprint Reported,Sprint Fixed
0,Ticket #1,UserStory #3,0.8,0.0,0.0,2,NOT_SET
1,Ticket #2,UserStory #10,1.8,0.0,0.0,2,NOT_SET
2,Ticket #3,UserStory #15,0.6,0.0,0.0,2,NOT_SET
3,Ticket #4,UserStory #22,0.6,0.0,0.0,2,NOT_SET
4,Ticket #5,UserStory #30,0.4,0.0,0.0,2,NOT_SET
5,Ticket #6,UserStory #36,2.0,0.0,0.0,2,NOT_SET


In [9]:
bugs_df.shape

(42, 7)

In [10]:
bugs_df['Estimated Cost'].sum()

45.00000000000001

In [11]:

stories_a = stories_df[stories_df['Team Id'] == 'Team A']
stories_a

,User Story Id,Original Estimate,Team Id,Developer,Product Manager,Percent Achieved,Planned,Sprint Planned,Sprint Delivered,Nb Open Bugs,Open Bugs,Nb Closed Bugs,Closed Bugs
0,UserStory #1,10,Team A,Beau Hockensmith,Sherlyn Cordle,1.0,True,1,1,1,[Ticket #15],0,[]
1,UserStory #2,10,Team A,Glenna Mcghie,Sherlyn Cordle,1.0,True,1,1,0,[],0,[]
2,UserStory #3,4,Team A,Francisco Hoppe,Edgar Hibbler,1.0,True,1,1,1,[Ticket #1],0,[]
3,UserStory #4,7,Team A,Beau Hockensmith,Sherlyn Cordle,1.0,True,1,1,1,[Ticket #16],0,[]
4,UserStory #5,3,Team A,Gregorio Darr,Edgar Hibbler,1.0,True,1,1,1,[Ticket #17],0,[]
5,UserStory #6,3,Team A,Beau Hockensmith,Sherlyn Cordle,1.0,True,1,1,1,[Ticket #36],0,[]
6,UserStory #7,8,Team A,Anton Easterday,Sherlyn Cordle,1.0,True,1,1,0,[],0,[]
7,UserStory #8,9,Team A,Anton Easterday,Edgar Hibbler,1.0,True,1,1,0,[],0,[]
8,UserStory #9,9,Team A,Gregorio Darr,Sherlyn Cordle,1.0,True,1,1,0,[],0,[]
9,UserStory #10,9,Team A,Gregorio Darr,Sherlyn Cordle,1.0,True,1,1,1,[Ticket #2],0,[]


In [ ]:
stories_a['Original Estimate'].sum()

In [ ]:
stories_a.groupby('Nb Open Bugs').count()

In [ ]:
def countUniques(seriesGroup):
    if (seriesGroup.unique()[0] == None):
        return 0
    else:
        return seriesGroup.unique().size

In [ ]:
countUniques(bugs_df['User Story Id'])

In [ ]:
bugs_df.shape

In [ ]:
modelsConfig.context.sprint

In [ ]:
def genReleaseCycleSheets(modelsConfig):
    RELEASE_DURATION = 150
    SPRINT_DURATION = 10
    
    tu_.loadTestResources()
    
    teams_df, stories_df, teamsRepo, storiesRepo, ticketsRepo = tu_.initTestData(tu_.DEV_DF, \
                                                                                           tu_.PM_DF, \
                                                                                           RELEASE_DURATION, \
                                                                                           SPRINT_DURATION, \
                                                                                          modelsConfig)
    
    NUMBER_OF_SPRINTS = 25

    entries_df, releaseLog = timecard.runReleaseCycle(teamsRepo, ticketsRepo, storiesRepo, datetime(2018, 1, 15), \
                                                   SPRINT_DURATION, NUMBER_OF_SPRINTS, modelsConfig)    
    
    return entries_df, releaseLog, teamsRepo, storiesRepo

In [ ]:
SPRINT_DURATION = 10
modelsConfig = ModelsConfig([DefaultCostModel(0.0)], [DistributedLagQualityModel()], GreedyAllocationModel()) 
modelsConfig.random.reset(271)
entries_df, releaseLog, TEAMS_REPO, STORIES_REPO = genReleaseCycleSheets(modelsConfig)

In [ ]:
tc_visuals.renderReleaseCycleLog('Team A', releaseLog, 1, 16)

In [ ]:
stories_df = build_stories_df(modelsConfig)
stories_df[:6]

In [ ]:
stories_df[stories_df['Nb Closed Bugs'] > 2]

In [ ]:
bugs = modelsConfig.context.ticketsRepo.getClosedTickets('UserStory #777')
bugs_df = build_bugs_df(bugs)
bugs_df

In [ ]:
entries_df[entries_df['User Story'] == 'UserStory #106']

In [ ]:
df = releaseLog.log['Team A'][9]['Outcome']
df

In [ ]:
dfe = releaseLog.log['Team A'][3]['planned_End_CURRENT_SPRINT']
dfe

In [ ]:
dfs = releaseLog.log['Team A'][3]['planned_Start_CURRENT_SPRINT']
dfs

In [ ]:
dfbl = releaseLog.log['Team A'][3]['backlog']
dfbl

In [ ]:
bugs_df = build_bugs_df(modelsConfig.context.ticketsRepo.tickets)
c1 = bugs_df['Sprint Reported'] < 4
c2 = bugs_df['Sprint Fixed'] > 2
b3 = bugs_df[c1]# & c2]
b3

In [ ]:
b3['User Story Id'].unique()

In [ ]:
b3.shape

In [ ]:
bugs_df.groupby('User Story Id').count()

In [ ]:
df = releaseLog.log['Team B'][6]['Outcome']
df[df['User Story Id'] == 'UserStory #120']

In [ ]:
bugs_df[bugs_df['User Story Id']=='UserStory #120']

In [ ]:
entries_df[entries_df['User Story'] == 'UserStory #120']

<h2>Burnout in release cycle - Pilot</h2>

In [ ]:
# 
def genReleaseCycle(modelsConfig):
    RELEASE_DURATION = 60
    SPRINT_DURATION = 10
    
    tu_.loadTestResources()
    
    teams_df, stories_df, teamsRepo, storiesRepo, ticketsRepo = tu_.initTestData(tu_.DEV_DF, \
                                                                                           tu_.PM_DF, \
                                                                                           RELEASE_DURATION, \
                                                                                           SPRINT_DURATION, \
                                                                                          modelsConfig)
    
    NUMBER_OF_SPRINTS = 25

    entries_df, worksheets = timecard.runReleaseCycle(teamsRepo, ticketsRepo, storiesRepo, datetime(2018, 1, 15), \
                                                   SPRINT_DURATION, NUMBER_OF_SPRINTS, modelsConfig)    
    
    return entries_df, worksheets, storiesRepo

In [ ]:
def genBurnout(entries_df, storiesRepo):
    bystory = entries_df.groupby('User Story')
    u = bystory.apply(storyInfo, storiesRepo)
    
    
    u = u.reset_index()
    u.drop(['level_1'], axis='columns', inplace=True)
    s = u.groupby('Final Sprint')
    burnout = s.apply(sprintInfo)
    burnout = burnout.reset_index()
    burnout.drop(['level_1'], axis='columns', inplace=True)
    

    return burnout, u

In [ ]:
def storyInfo(group_df, storiesRepo):
    info = {}
    
    #Go back, as min['Date'] is end of first sprint
    info['Start'] = [timecard.subtractBusinessDays(group_df['Date'].min(), SPRINT_DURATION)] 
    info['End'] = [group_df['Date'].max()]
    info['Elapsed Time'] = (info['End'][0] - info['Start'][0]).days
    info['Effort'] = group_df['Time Spent'].sum()
    info['Initial Sprint'] = [group_df['Sprint'].min()]
    info['Final Sprint'] = [group_df['Sprint'].max()]  
    '''
    userStoryId = group_df['User Story']
    userStory = storiesRepo.findStory(userStoryId)
    info['Original estimate'] = userStory.originalEstimate    
    '''
    
    return pd.DataFrame(info)

In [ ]:
def sprintInfo(group_df):
    info = {}
    
    info['Effort'] = group_df['Effort'].sum()
    info['Avg Effort'] = group_df['Effort'].mean()
    info['Stories Completed'] = [group_df['User Story'].count()]
    info['Over 1 sprint'] = [group_df[group_df['Elapsed Time']==14]['User Story'].count()]
    info['Over 2 sprint'] = [group_df[group_df['Elapsed Time']==28]['User Story'].count()]
    info['Over 3 sprint'] = [group_df[group_df['Elapsed Time']==42]['User Story'].count()]
    info['Over 4 sprint'] = [group_df[group_df['Elapsed Time']==56]['User Story'].count()]
 
    
    return pd.DataFrame(info)

In [ ]:
SPRINT_DURATION = 10
modelsConfig1 = ModelsConfig([DefaultCostModel(0.0)], [], BalancedAllocationModel(SPRINT_DURATION)) 
modelsConfig1.random.reset(271)
entries_df1, worksheets1, storiesRepo = genReleaseCycle(modelsConfig1)

In [ ]:
burn1, u1 = genBurnout(entries_df1, storiesRepo)
burn1

In [ ]:
u1

In [ ]:
SPRINT_DURATION = 10
modelsConfig2 = ModelsConfig([DefaultCostModel(0.0)], [], GreedyAllocationModel(SPRINT_DURATION)) 
modelsConfig2.random.reset(271)
entries_df2, worksheets2, storiesRepo = genReleaseCycle(modelsConfig2)

In [ ]:
burn2, u2 = genBurnout(entries_df2, storiesRepo)
burn2

In [ ]:
burn1['Effort'].plot(label = 'Balanced'), burn2['Effort'].plot(label = 'Greedy'), plt.legend()

In [ ]:
burn1['Avg Effort'].plot(label = 'Balanced'), burn2['Avg Effort'].plot(label = 'Greedy'), plt.legend()

In [ ]:
u1_10 = u1[u1['Final Sprint'] == 10]
u1_10.shape

In [ ]:
u1_10[u1_10['Initial Sprint'] == 10]['Effort'].mean()

In [ ]:
u1_10['Effort'].mean()

In [ ]:
u1_10[u1_10['Initial Sprint'] == 10].shape

In [ ]:
u1_10[u1_10['Initial Sprint'] == 9]['Effort'].mean()

In [ ]:
u1_10[u1_10['Initial Sprint'] == 9].shape